In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
import re

In [2]:
! pip install unidecode

In [3]:
def get_job_info(trabajo_link):
    """El input es un enlace que dirige a una única oferta de trabajo. Devuelve un tuple con la información más relevante: descripción del anuncio y del trabajo"""
    anuncio_list = []
    description_list = []
    import requests
    from bs4 import BeautifulSoup
    try:
        response = get_page(trabajo_link)
        result_page = BeautifulSoup(response.content,'lxml')
        anuncio_list = get_anuncio_list(result_page)
        description_list = get_job_description(result_page)
        empresa_name = get_name_empresa(result_page)
        output = unify_lists([empresa_name, anuncio_list, description_list])
        return output
    except:
        return None

In [4]:
def get_page(url):
    response = requests.get(url)
    if not response.status_code == 200:
        print("It didn't work")
    return response

In [5]:
def get_anuncio_list(result_page):
    '''This function recover the information concerning ID Empleo, Categoria, Ubicación, etc'''
    anuncio_list = []
    for info in result_page.find_all('div', class_='smallListingInfo'):
        if not 'Primera' in info.get_text(): #Exclude 'Primera Publicacion'
            if not 'Contrato' in info.get_text():
                anuncio_item = info.get_text()
                anuncio_item = anuncio_item[anuncio_item.find(':')+2:]
                anuncio_list.append(anuncio_item.replace('\n',''))  #Get items without \n
                #anuncio_list.append(info.get_text().replace('\n',''))
    return anuncio_list

In [6]:
def get_job_description(result_page):
    text = ''
    for paragraph in result_page.find_all('p'):
        if paragraph.text != "\n":
            text = text + paragraph.text
    text = clean_text(text)
    return text

In [7]:
def get_name_empresa(result_page):
    Enterprise_information = result_page.find('div', class_ = 'compProfileInfo')
    return Enterprise_information.find('strong').get_text()

In [8]:
def clean_text(text):
    import unidecode
    text = text.lower().replace('\n','') #Get items without \n
    text = re.sub("(\(\w+\))", '', text) #Delete parenthesis
    text = unidecode.unidecode(text) #Delete accents
    text = re.sub('[^A-Za-z0-9\s]','',text) #Delete anything but letters, numbers and spaces
    return text

In [9]:
def unify_lists(job_list):
    new_list = []
    new_list.append(job_list[0])
    for element in job_list[1]:
        new_list.append(element)
    new_list.append(job_list[2])
    return new_list

In [10]:
trabajo_link = "https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/695472/Analista-de-Tesorería.html?searchId=1545862649.53&page=1"
response = get_page(trabajo_link)
result_page = BeautifulSoup(response.content,'lxml')
anuncio_list = get_anuncio_list(result_page)
description_list = get_job_description(result_page)
empresa_name = get_name_empresa(result_page)
output = unify_lists([empresa_name, anuncio_list, description_list])

In [11]:
output

['NOSSAMODA SRL',
 '695472',
 'Santa Cruz de la Sierra, Bolivia',
 'Contabilidad y Economía ',
 'De Bs. 3.000 a Bs. 3.999 ',
 '28 Enero 2019',
 '27 Febrero 2019',
 'analista de tesoreria formacion titulado en auditoria financiera contaduria publica contaduria general o ramas afines experiencia y conocimientos  tres  anos de experiencia laboral en cargos de igual responsabilidad  elaboracion de conciliaciones bancarias y arqueos de caja elaboracion de flujos de caja gestion de pagos a proveedores locales y extranjeros seguimiento de cuentas pendientes de cobro y depositos de cobranzas del dia  preparacion de estados de cuentas por cobrar y pagar y control de emision de cheques registro contable de pagos a proveedores sueldos finiquitos etc  registro contable de cobros a clientes depositos transferencias etc  conocimiento en sistema contable sai   solidos manejo de windows office excel powerpoint outlook y word competencias  responsabilidad  organizacion  atencion a los detalles las pers

In [12]:
def trabajopolis(job_description):
    """ A partir de una (o más) palabras claves  """
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    for job in job_description:
        #url = "http://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D="+ job + "&page=1&listings_per_page=1000"
        url = "https://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D=cajero&page=1&listings_per_page=10"

        response = requests.get(url)
        if response.status_code == 200:
            print("The request response cycle was successful")
            results_page = BeautifulSoup(response.content,'lxml')
            #trabajo_list = list()
            trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
            for anuncio in trabajos:
                trabajo_link = anuncio.find('a').get('href')
                trabajo_name = anuncio.find('b').get_text()
                print(trabajo_link)
             #   trabajo_list.append((trabajo_name,trabajo_link))
            #for trabajo in trabajo_list:
                output = get_job_info(trabajo_link)
                trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
        else:
            print("The request response cycle work didn't work")
        time.sleep(10) 
    return trabajos_list

In [13]:
def trabajopolis2():
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    url = "https://www.trabajopolis.bo/search-results-jobs/?searchId=1546400657&action=search&page=1&listings_per_page=1500"
    response = requests.get(url)
    if response.status_code == 200:
        print("The request response cycle was successful")
        results_page = BeautifulSoup(response.content,'lxml')
        trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
        for anuncio in trabajos:
            trabajo_link = anuncio.find('a').get('href')
            trabajo_name = anuncio.find('b').get_text()
            print(trabajo_name)
            output = get_job_info(trabajo_link)
            trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
    else:
        print("The request response cycle work didn't work")
    return trabajos_list

In [14]:
Results = trabajopolis2()

The request response cycle was successful
Visitador(a) Médico(a) - Santa Cruz
Asistente de Coordinación Docente
Coordinador(a) de Talento Humano
Ejecutivo(a) de Ventas – Mercado Mutualista
Encargado(a) de Operaciones Terrestres - Santa Cruz
Agente Inmobiliario(a)
Product Manager - Santa Cruz
Gerente Comercial
Especialista Nacional en Media Tensión
Editor(a) Audiovisual - Santa Cruz
Pasante para el Área de Logística - Santa Cruz
Encargado(a) de Almacén y Despacho
Supervisor(a) de Ventas Canal Minorista - Santa Cruz
Administrador(a) de Taller Técnico
Técnico de Servicio - Potosí
Asistente Comercial - Trinidad
Cajero(a)
Analista de Distribución Secundaria y Operaciones
Residente de Obra
Ejecutivo(a) de Ventas - Idioma Chino Mandarín
Ejecutivo(a) de Ventas - Idioma Coreano
Ejecutivo(a) de Ventas
Cobrador(a)
Líder de Caja
Asistente Contable
Asistente de Compras y Servicios
Asesor(a) de Ventas Retail - Santa Cruz
Asistente de Oficina
Gestor(a) de Comunicación Institucional
Encargado(a) de Al

Jefe de Ventas - La Paz
Secretaria / Recepcionista
Ejecutivo(a) Cobranza de Campo - Cobija
Brand Activator Specialist - Fernet Branca
Ejecutivo(a) de Ventas Camiones y Buses - La Paz
Supply Analyst
Business and Supply Analyst
Logistics Analyst / Planner
Desarrollador(a)
Jefe de Transporte y Logística
Administrador(a) de Taller
Analista Legal
Ejecutivo(a) Comercial Externo(a) - Santa Cruz
Pasante de Servicio Técnico - Santa Cruz
Pasante de Servicio Técnico - Cochabamba
Auxiliar de Sistemas
Asesor(a) de Ventas PyME (Plazo Fijo)
Cobrador(a) - Cochabamba
Cobrador(a) - La Paz
Gestor(a) Comercial Sénior
Supervisor(a) de Producción - Santivañez
Analista Técnico
Jefe Técnico
Sub Gerente Técnico
Ejecutivo(a) de Reclamos
Asesor(a) de Créditos - Santa Rosa del Sara
Analista de Riesgo de Crédito PyME y Empresarial (Plazo Fijo) - Santa Cruz
Asistente Sala VIP - Santa Cruz
Contador(a) General
Contador(a)
Call Center - Plazo Fijo - Sucre
Call Center - Plazo Fijo - Tarija
Técnico Especialista Agropecu

Encargado(a) de Vehículos Oriente
Ejecutivo(a) de Ventas
Analista Comercial
Jefe de Inteligencia de Negocios
Jefe de Proyectos IT Sénior
Ejecutivo(a) de Ventas
Responsable - Responsabilidad Social Empresarial RSE
Auditor(a) Júnior - Santa Cruz
Ingeniero(a) de Proyectos
Coordinador(a) Comercial
Vendedor(a) - Santa Cruz
Analista de Revenue Management - Santa Cruz
Jefe de Administración
Asistente Comercial - Santa Cruz
Auxiliar de Almacenes - La Paz
Auxiliar de Almacenes - Santa Cruz
Técnico en Refrigeración
Jefe Regional de Atención al Cliente
Asistente de Importaciones
Asistente Administrativa(o)
Ingeniero(a) de Sistemas – Cochabamba
Encargado(a) de Administración de Sucursal - Santa Cruz
Auxiliar de Compras - Santa Cruz
Encargado(a) Comercial – Cochabamba
Jefe de Agencia - Tarija
Ejecutivo(a) de Negocios Banca Mediana Empresa - Cochabamba
Médico Laboral
Técnicos Especialistas en Mantenimiento de Radio Bases (Santa Cruz-Beni)
Asesores(as) Comerciales - La Paz / El Alto
Gerente de Crédit

In [15]:
len(Results)

634

In [16]:
import csv
# Open the file in the mode append and the encoding utf-8
outfile = open("/Users/Roberto/Documents/Python Scripts/Machine Learning Projects/Labor Market/TRABAJOPOLIS_FEB2019.csv", 'a',newline='', encoding = 'utf-8')
writer = csv.writer(outfile)
try:
    for i in range(len(Results)):
        csvRow = []
        # Transform the list shape:
        if Results[i][0] is not None:
            
            for cell in Results[i][0]:
                csvRow.append(cell)
                
            csvRow.append(Results[i][1])
            csvRow.append(Results[i][2])

            writer.writerow(csvRow)
finally:
    outfile.close() 

In [17]:
Results[400]

(['TIGO',
  '815551',
  'Potosí, Bolivia',
  'Marketing y Ventas ',
  'No Declarado ',
  '18 Diciembre 2019',
  '17 Enero 2020',
  'ejecutivo de ventas hogarsede potosi que te pedimos estudiante de 2do semestre en adelante en carreras de administracion de empresas ingenieria comercial finanzas ingenieria de sistemas ingenieria electronica psicologia yo ramas afines facilidad de palabra habilidades en atencion al cliente negociacion y cierre de ventas habilidades en ejecucion de estrategias comerciales ventas zonales institucionales deteccion de nichos y oportunidades de mercado capacidad de aprendizaje continuo conocimiento de microsoft office nivel basico disponibilidad de tiempo completo e inmediato competencias requeridas sentido de pertenencia orientacion a resultados organizacionales hacerse cargo  orientacion al cliente te ofrecemos ser parte de una de las companias lider en telecomunicaciones estabilidad laboral desarrollo profesional capacitacion constante ingresos de acuerdo a

In [17]:
Results[0]

(['Importante Empresa Comercial',
  '815978',
  'Santa Cruz de la Sierra, Bolivia',
  'Administración y Oficina ',
  'De Bs. 3.000 a Bs. 3.999 ',
  '16 Diciembre 2019',
  '15 Enero 2020',
  'auditor junior mision del cargorealizar auditorias a las areas designadas siguiendo la planificacion del departamento de auditoria elaborar los informes correspondientes de observaciones en la perspectiva de implementar procesos de mejoramiento de la gestion y la eficiencia requisitos experiencia minima de 1 ano en cargos similares de auditoria formacion en auditoria yo ramas afines manejo del office nivel intermedio manejo del sistema tr4 conocimiento de inventarios capacidad analitica trabajo bajo presion las personas interesadas que cumplan con el perfil podran remitir su curriculum vitae indicando su pretension salarial por trabajopolisbo'],
 'Auditor(a) Júnior',
 'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/815978/Auditor(a)-Júnior.html?searchId=1546400657&page=1')

In [18]:
Results = trabajopolis(['sql'])

The request response cycle was successful
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/815585/Cajero(a).html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/815434/Cajero(a).html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/815136/Cajero(a).html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/811069/Cajero(a)---Santa-Cruz.html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/810620/Cajero(a)---La-Paz.html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/810125/Cajero---Potosí.html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/809885/Cajero(a).html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/809309/Cajero(a)-y-Atención-al-Cliente

NameError: name 'time' is not defined

In [ ]:
hola = Results[1000:]

In [ ]:
for i in range(len(hola)-1):
    csvRow = []
        
    if Results[i][0] is not None:
            
        for cell in hola[i][0]:
            csvRow.append(cell)
                
        csvRow.append(hola[i][1])
        csvRow.append(hola[i][2])

print(csvRow)

In [53]:
csvrow = []
for cell in Results[i][0]:
    #print(cell)
    csvrow.append(cell)
csvrow.append(Results[i][1])
csvrow.append(Results[i][2])
print(csvrow)

['Guayabochi srl', '699870', 'Santa Cruz de la Sierra, Bolivia', 'Administración y Oficina ', 'No Declarado ', '07 Enero 2019', '06 Febrero 2019', 'empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo', 'Mensajero(a)', 'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/699870/Mensajero(a).html?searchId=1546400657&page=1']


In [43]:
Results[i][0]

['Guayabochi srl',
 '699870',
 'Santa Cruz de la Sierra, Bolivia',
 'Administración y Oficina ',
 'No Declarado ',
 '07 Enero 2019',
 '06 Febrero 2019',
 'empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo']

In [44]:
for cell in Results[i][0]:
    print(cell)

Guayabochi srl
699870
Santa Cruz de la Sierra, Bolivia
Administración y Oficina 
No Declarado 
07 Enero 2019
06 Febrero 2019
empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo


In [45]:
csvRow.append(cell)